# Global Imports

In [45]:
import os
from pprint import pprint

from dotenv import find_dotenv, load_dotenv

load_dotenv(find_dotenv())

True

We are using `load_dotenv(find_dotenv())` to find and load `.env` file. <br>
Why? 🤔 <br> 
If your environemnt file is not located in the root of your project we can still access it. <br>

# LLMs
LLMs [documentation](https://python.langchain.com/docs/modules/model_io/llms).

In [42]:
from langchain_openai import OpenAI, ChatOpenAI
from langchain_core.messages import HumanMessage

In [73]:
llm = OpenAI(
    api_key=os.environ["OPENAI_API_KEY"], organization=os.environ["OPENAI_ORGANIZATION"]
)
chat_model = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    organization=os.environ["OPENAI_ORGANIZATION"],
    model="gpt-3.5-turbo",
)

The LLM objects take string as input and output string. <br>
The ChatModel objects take a list of messages as input and output a message. <br>
For a deeper conceptual explanation of this difference please see [this documentation](https://python.langchain.com/docs/modules/model_io/concepts). <br>

In [74]:
text = "Are you an Alien?"
messages = [HumanMessage(content=text)]

response = llm.invoke(text)
pprint(f"LLM response: {response}")
pprint(f"LLM response type: {type(response)}")

pprint("-" * 100)

response = chat_model.invoke(messages)
pprint(f"Chat Model response: {response}")
pprint(f"Chat Model response type: {type(response)}")

'LLM response: \n\nNo, I am a digital AI created by humans.'
"LLM response type: <class 'str'>"
'----------------------------------------------------------------------------------------------------'
("Chat Model response: content='No, I am an artificial intelligence created by "
 "humans to assist with answering questions and providing information.' "
 "response_metadata={'token_usage': {'completion_tokens': 19, 'prompt_tokens': "
 "12, 'total_tokens': 31}, 'model_name': 'gpt-3.5-turbo', "
 "'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': "
 'None}')
"Chat Model response type: <class 'langchain_core.messages.ai.AIMessage'>"


The problem with raw LLMs is that they don’t remember the history of the conversations

In [50]:
response = chat_model.invoke("What was my previous question?")
pprint(response)

AIMessage(content='Your previous question was "What is your favorite color?"', response_metadata={'token_usage': {'completion_tokens': 11, 'prompt_tokens': 13, 'total_tokens': 24}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})


# Chains
Chains [documentation](https://python.langchain.com/docs/modules/chains).

## Add memory manually 
Following their current documentation and supported modules we can create a memory modeul and add history manually

In [98]:
from operator import itemgetter

from langchain.globals import set_debug, set_verbose
from langchain.memory import ConversationBufferMemory
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain_core.runnables import RunnableLambda, RunnablePassthrough

To see chain's process of toughts we neeed to activate **debugging** mode and **verbose**.
> 📎 **Note**: For some reason **verbose** is not working so we can use debug mode. Debug mode outputs a lot of text that is not important to us now so we will continu using basic output.

In [129]:
set_debug(value=False)
set_verbose(value=True)

In [122]:
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", "You are a helpful chatbot"),
        MessagesPlaceholder(variable_name="history"),
        ("human", "{input}"),
    ]
)

In [123]:
memory = ConversationBufferMemory(return_messages=True)
memory.load_memory_variables({})

{'history': []}

In [124]:
chain = (
    RunnablePassthrough.assign(
        history=RunnableLambda(memory.load_memory_variables) | itemgetter("history")
    )
    | prompt
    | chat_model
)

In [125]:
inputs = {"input": "Are you an Alien?"}

response = chain.invoke(inputs)
response

AIMessage(content='No, I am not an alien. I am a computer program designed to assist and provide information to users. How can I help you today?', response_metadata={'token_usage': {'completion_tokens': 29, 'prompt_tokens': 22, 'total_tokens': 51}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

In [126]:
memory.save_context(inputs, {"output": response.content})

In [127]:
memory.load_memory_variables({})

{'history': [HumanMessage(content='Are you an Alien?'),
  AIMessage(content='No, I am not an alien. I am a computer program designed to assist and provide information to users. How can I help you today?')]}

In [128]:
chain.invoke({"input": "What was my previous question?"})

AIMessage(content='Your previous question was "Are you an Alien?" Is there anything else you would like to ask or discuss?', response_metadata={'token_usage': {'completion_tokens': 22, 'prompt_tokens': 65, 'total_tokens': 87}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': 'fp_3bc1b5746c', 'finish_reason': 'stop', 'logprobs': None})

## Add memory automatically
We can use chain class that support automatic history tracking but they may be unsupported in the future.

Here we can set `verbose` in chain directly and it will work.

IMO this approach is better.

In [130]:
from langchain_openai import ChatOpenAI
from langchain.chains import ConversationChain

In [132]:
chat_model = ChatOpenAI(
    api_key=os.environ["OPENAI_API_KEY"],
    organization=os.environ["OPENAI_ORGANIZATION"],
    temperature=0,
)

conversation = ConversationChain(
    llm=chat_model, verbose=True, memory=ConversationBufferMemory()
)

In [133]:
conversation.predict(input="Are you an Alien?!")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:

Human: Are you an Alien?!
AI:

> Finished chain.


'No, I am not an alien. I am an artificial intelligence created by humans to assist with various tasks and provide information. I do not have a physical form like aliens are often depicted as having.'

As part of the prompt, we see now that there is a bit more then our simple prompt. <br>
We have a system prompt, the history of the conversation, and the human’s question. <br>
The system prompt allows us to give the LLM more context on what needs to be done. <br>
Let’s see if it remembers: <br>

In [134]:
conversation.predict(input="What was my previous question?")



> Entering new ConversationChain chain...
Prompt after formatting:
The following is a friendly conversation between a human and an AI. The AI is talkative and provides lots of specific details from its context. If the AI does not know the answer to a question, it truthfully says it does not know.

Current conversation:
Human: Are you an Alien?!
AI: No, I am not an alien. I am an artificial intelligence created by humans to assist with various tasks and provide information. I do not have a physical form like aliens are often depicted as having.
Human: What was my previous question?
AI:

> Finished chain.


'Your previous question was "Are you an Alien?!"'

# Prompt Templates